# DIA Analysis: Making light curves from DIA Source data
Steven Rodney and the DIA team of the 2020 Stack Club Course
Last Verified to Run: TBD

copying mercilessly from WMWV's notebook: 
https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/dia_source_object_stamp.ipynb

and several Bryce Kalmbach nbs:
https://github.com/LSSTScienceCollaborations/StackClub/blob/master/Measurement/twinkles_light_curves.ipynb
https://github.com/LSSTScienceCollaborations/StackClubCourse/tree/master/Session03

see also the other reference nbs in this subdir of the StackClub
After completing this Notebook, the users will be able to


1. Select and plot the light curve of a DIA Object.
2. Display and inspect postage stamps of a selected DIAObject.

In [3]:
%matplotlib widget

### TODO : remove un-needed imports

In [2]:
# Inject gcr-catalogs that supports DIA source into path.
import os
import math
import sys

import numpy as np
import pandas as pd

from astropy.coordinates import SkyCoord
import astropy.units as u

In [7]:
import lsst.afw.display as afwDisplay
from lsst.afw.math import Warper
import lsst.afw.geom as afwGeom
from lsst.daf.persistence import Butler
from lsst.geom import SpherePoint
import lsst.geom

In [8]:
import lsst.afw.display as afw_display
afw_display.setDefaultBackend('matplotlib')

In [9]:
import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lsst.geom

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time

#%matplotlib widget
%matplotlib ipympl
#%matplotlib inline

first, point the butler to a repo with diff images (from stack club session 4)

In [10]:
data_dir = '/project/stack-club/decam_hits_2015_subset/'
butler = dafPersist.Butler(data_dir)

We know there are difference images in this dataset, but what are the actual datasets called when we access them through the butler? We can remind ourselves with `getDatasetTypes`. 

In [12]:
data_types = butler.getDatasetTypes()
diff_data_types = [x for x in data_types if x.startswith('deepDiff') ]

Use the queryMetadata to find what diff images are available.

In [14]:
butler.queryMetadata('deepDiff_differenceExp', ('visit', 'ccdnum'), dataId={'filter': 'g'})[:10]

[(410915, 1),
 (410915, 2),
 (410915, 3),
 (410915, 4),
 (410915, 5),
 (410915, 6),
 (410915, 7),
 (410915, 8),
 (410915, 9),
 (410915, 10)]

Select one of these at random

In [15]:
visit_num = 410915
ccd_num = 6

In [18]:
# Load a calexp image and get the maskedImage object and the source catalog
calexp = butler.get('calexp', {'visit': visit_num, 'ccdnum': ccd_num, 'filter': 'g'})
calexp_im = calexp.maskedImage
calexp_src_cat = butler.get('src', {'visit': visit_num, 'ccdnum': ccd_num, 'filter': 'g'})

In [38]:
# Now load the corresponding diff image, and get the same maskedImage and source catalog
diffexp = butler.get('deepDiff_differenceExp', {'visit': visit_num, 'ccdnum': ccd_num, 'filter':'g'})
diffexp_im = diffexp.getMaskedImage()
diffexp_src_cat = butler.get('deepDiff_diaSrc', {'visit': visit_num, 'ccdnum': ccd_num, 'filter': 'g'})

### Get Visit Info
Since we want to get light curves in the end we'll need to learn how to get the time of the visits. Let's use `getInfo` found in the `Exposure` object.

In [21]:
# extract the visit info
exp_visit_info = calexp.getInfo().getVisitInfo()

# get the observation date
visit_date_python = exp_visit_info.getDate().toPython()

# convert to MJD
visit_date_astropy = Time(visit_date_python)
print(visit_date_astropy.mjd)

57070.141865742786


# find a bright source in the diff image
(when we connect these NBs together, this will be one of the planted fake sources)

In [56]:
from bokeh.io import output_notebook, show 
from bokeh.layouts import gridplot
from bokeh.plotting import figure as bokehfigure
from bokeh.models import ColumnDataSource, Range1d, HoverTool, Selection

# Need this line to display bokeh plots inline in the notebook
output_notebook()

Loading BokehJS ...

In [66]:
diffexp_src_cat_astropy = diffexp_src_cat.asAstropy()

In [75]:
ra = diffexp_src_cat_astropy['coord_ra']
dec = diffexp_src_cat_astropy['coord_dec']

psfflux = diffexp_src_cat_astropy['slot_PsfFlux_instFlux']
psffluxerr = diffexp_src_cat_astropy['slot_PsfFlux_instFluxErr']
sig2noise = psfflux / psffluxerr

plotdata = ColumnDataSource(data=dict(
    ra=ra/np.cos(np.median(dec)), dec=dec, 
    f=psfflux, df=psffluxerr, sn=sig2noise,
    srcidx=diffexp_src_cat_astropy['id']))

In [76]:
# Create a custom hover tool on both panels
hover_left = HoverTool(tooltips=[("(RA,DEC)", "(@ra, @dec)"),
                                 ("srcidx", "@srcidx")])
hover_right = HoverTool(tooltips=[("S/N", "@sn"),
                                  ("srcidx", "@srcidx")])
TOOLS = "box_zoom,box_select,lasso_select,reset,help"
TOOLS_LEFT = [hover_left, TOOLS]
TOOLS_RIGHT = [hover_right, TOOLS]

In [77]:
TOOLS = "box_zoom,box_select,lasso_select,reset"

# More attractive colors
kwargs = {'selection_fill_color':'steelblue', 
          'selection_line_color': 'steelblue',
          'nonselection_fill_color': 'silver', 
          'nonselection_line_color': 'silver'}

# RA vs Dec
left = figure(tools=TOOLS_LEFT, plot_width=300, plot_height=300, title=None)
left.circle('ra', 'dec', source=plotdata, **kwargs)
left.xaxis.axis_label = 'R.A.'
left.yaxis.axis_label = 'Dec.'

# Flux vs Fluxerr
right = figure(tools=TOOLS_RIGHT, plot_width=300, plot_height=300, title=None)
right.circle('f', 'df', source=plotdata, **kwargs)
right.xaxis.axis_label = 'flux'
right.yaxis.axis_label = 'fluxerr'
#right.x_range= Range1d(-50, 2000)
#right.y_range= Range1d(-50, 600)

p = gridplot([[left, right]])

show(p)

In [36]:
# For now, just choose a random index, requiring that it has S/N > 10 (nice and bright, we hope?)
ibright = np.where(sig2noise>10)[0]

# pick the first source in the list of bright sources
idx = ibright[0]

(array([  8,   9,  14,  15,  18,  27,  32,  35,  46,  62,  63,  78,  79,
         80,  85,  88,  94,  95, 100, 101, 106, 112, 126]),)

## END OF SINGLE-EPOCH WORK

Below here is where we work across epochs to make a light curve

### TODO : figure out the RA,Dec of this object

### TODO : find matching objects in the other diff images 
 Actually, what we really want to do here is to force the construction of a new DIAObject catalog, using the diff images that now have fake PSFs planted into them.   This should be possible, but will require some digging to figure out exactly how.   Good extension of this StackClub project!!

tip from Bryce Kalmbach: 

https://pipelines.lsst.io/py-api/lsst.afw.table.MultiMatch.html#lsst.afw.table.MultiMatch

### TODO : Plot the light curves of objects from the DIAObject catalog